# Dependencies

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Types of Spaces

In [7]:
# 0, 1, 2
Discrete(3).sample()

2

In [10]:
# An array 3 x 3 (an array that contains 3 arrays with each array containing 3 values)
# 0 and 1 are the lower and upper values (the range of where the values will lie)
Box(0,1,shape=(3,3)).sample()

array([[0.11172476, 0.13338836, 0.675468  ],
       [0.09634547, 0.6680662 , 0.38782513],
       [0.8826869 , 0.14965059, 0.20709646]], dtype=float32)

In [25]:
# Wrapper 1 that brings spaces together
Tuple((Discrete(3), Box(0,1,shape=(3,3)), MultiBinary(4))).sample()

(1,
 array([[0.72381985, 0.75307155, 0.836142  ],
        [0.75173223, 0.91596746, 0.18010943],
        [0.66017056, 0.07019453, 0.7298493 ]], dtype=float32),
 array([1, 0, 0, 1], dtype=int8))

In [20]:
# Wrapper 2 that brings spaces together
Dict({'height':Discrete(2), 'speed':Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([19.978773], dtype=float32))])

In [17]:
# Contains 4 positions
MultiBinary(4).sample()

array([1, 1, 0, 1], dtype=int8)

In [19]:
MultiDiscrete([5,2,2]).sample()

array([2, 1, 0], dtype=int64)

# Building Environment
- Build an agent to give us the best shower possible
- Random temperature
- Agent does not know we prefer the temperature to be between 37 degrees to 39 degrees

In [26]:
# Tap off, tap on, or remains unchanged
Discrete(3)

Discrete(3)

In [32]:
# First way of defining a box space
Box(low=np.array([0]), high=np.array([100])).sample()

array([92.47772], dtype=float32)

In [36]:
# Second way of defining a box space
Box(low=0, high=100, shape=(1,)).sample()

array([1.2043661], dtype=float32)

In [43]:
class ShowerEnv(Env):
    
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        # Our temperature
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
    
    def step(self, action):
        # Apply temperature adjustment
        self.state += action-1
        # Decrease shower time
        self.shower_length -= 1
        
        # Calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            # Reward can be 0 as well
            reward = -1
            
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        # Implement viz (optional)
        pass
    
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        
        return self.state

In [40]:
# 0: Decreasing shower temp by 1 deg
# 1: No change
# 2: Increasing shower temp by 1 deg
# Based on the action, our state will change to be
# As an example: 1 0 would be no change to the temperature
value = Discrete(3).sample()
print(value, value-1)

1 0


In [44]:
env = ShowerEnv()

In [48]:
env.observation_space.sample()

array([66.39811], dtype=float32)

In [49]:
env.action_space.sample()

2

# Testing the Environment

In [53]:
episodes = 5

for episode in range(1, episodes+1):
    observation = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        score += reward
    print('Episode {}, Score: {}'.format(episode, score))
env.close()

Episode 1, Score: -56
Episode 2, Score: -36
Episode 3, Score: -26
Episode 4, Score: 16
Episode 5, Score: 10


# Train the Model

In [54]:
log_path = os.path.join('Training', 'Logs')

In [55]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [58]:
# Can run this piece of code everytime on the same model and it keeps learning ; no need for another instantiation of it
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_10
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -2.94    |
| time/              |          |
|    fps             | 910      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -2.85        |
| time/                   |              |
|    fps                  | 569          |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0045064967 |
|    clip_fraction        | 0.0569       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.733       |
|    explained_variance   

# Save the Model

In [59]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [60]:
model.save(shower_path)

In [61]:
del model

In [62]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [66]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(-24.0, 54.99090833947008)